In [69]:
import nltk
from sklearn_crfsuite import CRF
from nltk.corpus.reader import ConllChunkCorpusReader
import re
from sklearn.model_selection import train_test_split
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics._classification import classification_report
from sklearn.naive_bayes import MultinomialNB
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pickle

In [70]:
sentenceNtag = []

In [71]:
with open("D:/PythonProject/CRF_tokenize/IOB_data/train1.txt", encoding='utf8') as file:
    for line in file:
        cnt = 0
        word_buff = ""
        tag_buff = ""
        
        while line[cnt].isspace() != True:
            word_buff += line[cnt]
            cnt += 1
        
        cnt = len(line) - 2 
        while line[cnt].isspace() != True: 
            tag_buff += line[cnt]
            cnt -= 1
        tag_buff = tag_buff[::-1]
        
        sentenceNtag.append((word_buff, tag_buff))


In [72]:
len(sentenceNtag)

164787

In [73]:
sentences = []

temp = []
for i in range(len(sentenceNtag)):
    if sentenceNtag[i][0] != '.':
        temp.append(sentenceNtag[i])
    else:
        sentences.append(temp)
        temp = []

In [74]:
print(len(sentences))

5367


In [75]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 1:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-2:word.lower()': sent[i-2][0].lower(),
            '-2:word.istitle()': sent[i-2][0].istitle(),
            '-2:word.isupper()': sent[i-2][0].isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-2:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+2:word.lower()': sent[i+2][0].lower(),
            '+2:word.istitle()': sent[i+2][0].istitle(),
            '+2:word.isupper()': sent[i+2][0].isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [76]:
%%time
X_train = [sent2features(s) for s in sentences]
y_train = [sent2labels(s) for s in sentences]

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=None)


CPU times: total: 656 ms
Wall time: 653 ms


In [64]:
X_train[0]

[{'bias': 1.0,
  'word.lower()': '',
  'word[-3:]': '',
  'word[-2:]': '',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': 'bao',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+2:word.lower()': 'giờ',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'bao',
  'word[-3:]': 'Bao',
  'word[-2:]': 'ao',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': 'giờ',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+2:word.lower()': 'cũng',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': 'giờ',
  'word[-3:]': 'giờ',
  'word[-2:]': 'iờ',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'bao',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-2:word.lower()': '',
  '-2:word.istitle()'

In [84]:
%%time

crf = CRF(algorithm = "lbfgs",
          c1 = 0.1, 
          c2 = 0.1, 
          max_iterations= 100, 
          all_possible_transitions=True)
try:
    crf.fit(X_train, y_train)
except:
    pass

CPU times: total: 9.44 s
Wall time: 9.52 s


In [85]:
labels = list(crf.classes_)
labels.remove('')
labels

['B_W', 'O', 'I_W']

In [86]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.934158175254227

In [68]:
import pickle

pickle.dump(crf, open('crf_model.pkl', 'wb'))

In [14]:
with open('crf_model.pkl', 'rb') as file:
    crf_model = pickle.load(file)
    

In [21]:
text = input()
text = text.split(" ")
text = [(word, -1) for word in text]
untag_text = []
untag_text.append(sent2features(text))
prediction = crf_model.predict(untag_text)
tagged_text = []

for i in range(len(prediction[0])):
    tagged_text.append((text[i][0], prediction[0][i]))

tagged_text

[('Thuế', 'B_W'),
 ('thu', 'B_W'),
 ('nhập', 'I_W'),
 ('cá', 'B_W'),
 ('nhân', 'I_W')]

In [22]:
token_list = []
pre_word = tagged_text[0][0]

for i in range(1, len(tagged_text)):
    if tagged_text[i][1] == "I_W":
        pre_word += "_" + tagged_text[i][0]
    else:
        token_list.append(pre_word)
        pre_word =  tagged_text[i][0]

token_list.append(pre_word)

token_list

['Thuế', 'thu_nhập', 'cá_nhân']